<h1>1.读取认证信息和租户信息</h1>
认证信息不需要在每次调用接口时都去通过登录重新获取，认证信息的有效时间是7天，因此在7天内都可以重复使用，可以直接读取user_info.json中保存的认证信息。

In [ ]:
import json

with open('user_info.json', 'r', encoding='utf-8') as f:
    payload = json.load(f)

token=payload["token"]
tenantId=payload["tenantId"]
print("认证信息：",token)
print("租户信息：",tenantId)

<h1>2.上传设备数据</h1>
在上传前需要将资产、设备、指标数据按照模板格式整理到excel中，带*号的列为必填项。</br>
对于实测数据的接入来说，可以只上传设备和指标信息，因为实测数据实际上是对应的某个设备的某个指标数据，因此只需要上传这两种数据就可以接入实测了。资产相关的信息可以选择不填写。</br>
例如示例文件"iot/data/资产映射.xlsx"中已经设置了三个设备，每个设备都有两个指标，分别是Discharge和WaterLevel，在后续推送实测数据时也会用到这两个指标名称。同时也会用到导入设备后自动生成的设备ID

In [ ]:
import os
from dhicn_iot_service import *

configuration=Configuration.get_default_copy()
configuration.host="https://online-products.dhichina.cn/gateway/iot-service" #服务地址
api_client=ApiClient(configuration)

api_client.set_default_header('Authorization',token)
api_client.set_default_header('tenantId',tenantId)

# 调用接口
param='data\\资产配置.xlsx'

iot_instance=IntegrationApi(api_client)
response=iot_instance.api_v1_iot_integration_import_iot_config_excel_post(excel_file=param)

print("result:", response)
if response["status"] == False:
    os._exit()



<h1>3.获取设备数据</h1>
数据上传成功后，可以通过下方接口获取，会返回设备列表及设备下的所有指标信息，在之后的http、opcua接入实测数据的案例中，都需要先通过这个接口获取到设备数据，然后设置到参数中。

In [ ]:
import os
from dhicn_iot_service import *

configuration=Configuration.get_default_copy()
configuration.host="https://online-products.dhichina.cn/gateway/iot-service" #服务地址
api_client=ApiClient(configuration)
api_client.set_default_header('Authorization',token)
api_client.set_default_header('tenantId',tenantId)

iot_instance=DeviceApi(api_client)
param={
    'page_index':1,
    'page_size':1000
}
response = iot_instance.api_v1_iot_device_list_get(**param)
print("result:", response)

#读取设备id
print("设备id:", response["data"]["list"][0]["id"])

if response["status"] == False:
    os._exit()